In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.hierarchical_portfolio import HRPOpt
warnings.filterwarnings('ignore')

from AA import DataDownloader, AssetAllocation

In [ ]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA'] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065
ff_factors_expectations = {'Mkt-RF': 0.05, 'SMB': 0.02, 'HML': 0.03, 'RF': 0.02}

asset_prices, benchmark_prices, ff_factors = downloader.download_data(start_date=start_date, end_date=end_date,
                                                                      assets=assets, benchmark=benchmark)

# Default limits for every asset (min 1% - max 100%):  boounds = tuple((0.01, 1) for _ in range(self.num_assets))
Asset_allocation = AssetAllocation(asset_prices=asset_prices, benchmark_prices=benchmark_prices, rf=rf, ff_factors=ff_factors) 
Asset_allocation.calculate_ff_expected_returns(ff_factors_expectations)

In [ ]:
# Set Black-Litterman Expectations (Optional) 

P = np.array([
    [1, 0, 0, -1, 0],  
    [0, 1, -1, 0, 0],
    [0, 0, 0, 0, 1],
])

Q = np.array([0.05,  
              0.03,
              -0.15])  

Omega = np.diag([0.1**2, 0.15**2, 0.08**2])  # Incertidumbre en las vistas
tau = 0.08  # Incertidumbre en los rendimientos de equilibrio

Asset_allocation.set_blacklitterman_expectations(P, Q, tau, Omega)

### RMT Filter

In [ ]:
#Matriz original de covarianzas
Asset_allocation.asset_cov_matrix

In [ ]:
#
eigenvalues, eigenvectors = np.linalg.eigh(Asset_allocation.asset_cov_matrix)
eigenvalues, eigenvectors

In [ ]:
T, N = Asset_allocation.asset_returns.shape
T, N

In [ ]:
sigma_squared = np.mean(np.diag(Asset_allocation.asset_cov_matrix))
sigma_squared 

In [ ]:
lambda_plus = sigma_squared * (1 + np.sqrt(N/T))**2
lambda_plus

In [ ]:
filtered_eigenvalues = np.clip(eigenvalues, 0, lambda_plus)
filtered_eigenvalues

In [ ]:
filtered_eigenvalues - eigenvalues

In [ ]:
# Matriz filtrada=Eigenvectores×Matriz Diagonal de Eigenvalores Filtrados×Eigenvectores 

filtered_cov_matrix = eigenvectors @ np.diag(filtered_eigenvalues) @ eigenvectors.T
pd.DataFrame(filtered_cov_matrix)

### Asset Allocation

In [ ]:
optimizations = Asset_allocation.Optimize_Portfolio(method = "SLSQP")
optimizations

Usando el mismo timeframe y activos calculamos las metricas con los pesos datos por la optimizacion y verificamos el valor del porceso manual y la class AssetAllocation

### Sharpe 

In [ ]:
Weights_Sharpe, Value_Sharpe = optimizations.loc["Max Sharpe"][:-1], optimizations.loc["Max Sharpe"][-1]
weights = Weights_Sharpe

#retornos de activos
returns = asset_prices.pct_change().dropna()
returns.head()

In [ ]:
#retornos del portafolio
portfolio_returns = returns.dot(weights)
portfolio_returns.head()

In [ ]:
#retornos menos la tasa libre de riesgo diaria
excess_returns_daily = portfolio_returns - (rf / 252)
excess_returns_daily


In [ ]:
# Anualizamos el promedio  de la resta para obtener rendimiento promedio anual del portafolio
excess_returns_annualized = excess_returns_daily.mean() * 252
excess_returns_annualized

In [ ]:
# Calculamos volatilidad anual del portafolio
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
portfolio_volatility 

In [ ]:
# Calculamos Sharpe
sharpe_ratio = excess_returns_annualized / portfolio_volatility
sharpe_ratio

In [ ]:
# Comparamos
round(sharpe_ratio - Value_Sharpe, 10)

**Diferencia menor a $ 1e^{-10}$**

### Omega

In [ ]:
Weights_Omega, Value_Omega = optimizations.loc["Max Omega"][:-1], optimizations.loc["Max Omega"][-1]
weights = Weights_Omega

weights

In [ ]:
# Retornos de activos
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

In [ ]:
# Calcular los retornos diarios del benchmark
benchmark_returns = benchmark_prices.pct_change().dropna()  
benchmark_returns 

In [ ]:
# Obtener diferencia del portafolio respecto al benchmark
excess_returns = portfolio_returns[0] -  benchmark_returns[benchmark_returns.columns[0]] 
excess_returns

In [ ]:
positive_excess = excess_returns[excess_returns > 0].sum()
negative_excess = -excess_returns[excess_returns < 0].sum()
positive_excess, negative_excess

In [ ]:
omega_ratio = positive_excess / negative_excess
omega_ratio

In [ ]:
# Comparamos
round(omega_ratio - Value_Omega, 10)

**Diferencia menor a $ 1e^{-10}$**

### Safety First Ratio

In [ ]:
Weights_SFRatio, Value_SFRatio = optimizations.loc["Safety-First"][:-1], optimizations.loc["Safety-First"][-1]
weights = Weights_SFRatio

weights

In [ ]:
# Retorno del portafolio
returns = asset_prices.pct_change().dropna()
portfolio_returns = pd.DataFrame(returns.dot(weights)) 
portfolio_returns

In [ ]:
# Retorno del portafolio
portfolio_return = portfolio_returns.mean().item()
portfolio_return

In [ ]:
# Retorno Mínimo Aceptable (MAR)
MAF = rf / 252
MAF

In [ ]:
# Volatilidad del portafolio
cov_matrix = returns.cov()
portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
portfolio_vol

In [ ]:
# Calculamos el SFratio
SFratio = (portfolio_return - MAF) / portfolio_vol

print(f"Safety First Ratio: {SFratio}")

In [ ]:
# Comparamos
round(SFratio - Value_SFRatio, 10)

**Diferencia menor a $ 1e^{-10}$**

### HRP

In [ ]:
Weights_HRP = optimizations.loc["HRP"][:-1]
weights = Weights_HRP
weights

In [ ]:
# Obtener pesos usando PyPortfolioOpt
hrp_pypfopt = HRPOpt(returns)
weights_pypfopt = hrp_pypfopt.optimize()
weights_pypfopt = pd.Series(hrp_pypfopt.clean_weights())

In [ ]:
# DataFrame para comparar
comparison_df = pd.DataFrame({
    'Weights_HRP_Implementation': weights,
    'Weights_HRP_PyPortfolioOpt': weights_pypfopt
})
comparison_df = comparison_df.fillna(0)

In [ ]:
# Calcular las diferencias
comparison_df['Differences'] = round(comparison_df['Weights_HRP_Implementation'] - comparison_df['Weights_HRP_PyPortfolioOpt'], 5)
comparison_df

**Diferencia menor a $ 1e^{-5}$**

### Semivariance

In [ ]:
Weights_Semivariance, Value_Semivariance = optimizations.loc["Semivariance"][:-1], optimizations.loc["Semivariance"][-1]
weights = Weights_Semivariance

weights

In [ ]:
# Retorno
returns = asset_prices.pct_change().dropna()
asset_cov_matrix = returns.cov()

In [ ]:
# Calculamos Semivarianza
semivariance = np.dot(weights, np.dot(asset_cov_matrix, weights))
print(f"Semivariance: {semivariance}")

In [ ]:
# Comparamos
round(semivariance - Value_Semivariance, 10)

**Diferencia menor a $ 1e^{-10}$**